In [ ]:
%load_ext watermark
%watermark  -d -u -a 'Andreas Mueller, Kyle Kastner, Sebastian Raschka' -v -p numpy,scipy,matplotlib,scikit-learn

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# 监督学习(Supervised Learning) Part 2 -- 回归分析(Regression Analysis)

在回归中，我们试图预测一个连续的输出变量——与我们在前面的分类例子中预测的标称变量不同。

从一个简单的例子开始，它有一个特征维度(说明变量)和一个目标变量。从带有一些噪声的正弦曲线中创建一个数据集:

In [ ]:
x = np.linspace(-3, 3, 100)
print(x)

In [ ]:
rng = np.random.RandomState(42)
y = np.sin(4 * x) + x + rng.uniform(size=len(x))

In [ ]:
plt.plot(x, y, 'o');

线性回归(Linear Regression)
=================

第一个模型，是所谓的简单线性回归。在这里，我们想让数据拟合出一条直线，最简单的线性模型，就是简单地试图在直线上预测数据。找到这样一条线的一种方法，是线性回归`LinearRegression`(也称为[*普通最小二乘法(OLS)*](https://en.wikipedia.org/wiki/Ordinary_least_squares)回归)。线性回归的接口，和之前的分类器完全一样，只是``y``现在包含浮点值，而不是分类标签。

之前提到过，scikit-learn API要求我们以一维数组的形式提供目标变量(`y`);scikit-learn API要求样本(`X`)以二维数组形式提供——就算只包含一个特征。因此，将一维的`x`NumPy数组，转换成具有两个轴的`X`数组:

In [ ]:
print('Before: ', x.shape)
X = x[:, np.newaxis]
print('After: ', X.shape)

和之前一样，将数据集分成训练集(75%)和测试集(25%):

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

接下来，用在`LinearRegression`中实现的学习算法**将回归模型拟合(fit)到训练数据**:

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

对训练数据拟合完成后，就得到了用以下值对线性回归模型进行参数化的模型：

In [ ]:
print('Weight coefficients: ', regressor.coef_)
print('y-axis intercept: ', regressor.intercept_)

由于回归模型是线性的，目标变量(y)和特征变量(x)之间的关系定义为

$$y = weight \times x + \text{intercept}$$.

将最小值和最大值代入方程，可以绘制出拟合到训练数据的回归:

In [ ]:
min_pt = X.min() * regressor.coef_[0] + regressor.intercept_
max_pt = X.max() * regressor.coef_[0] + regressor.intercept_

plt.plot([X.min(), X.max()], [min_pt, max_pt])
plt.plot(X_train, y_train, 'o');

与之前notebook中分类用的估计类似，用`predict`方法来预测目标变量。我们期望预测值落在之前绘制的直线上:

In [ ]:
y_pred_train = regressor.predict(X_train)

In [ ]:
plt.plot(X_train, y_train, 'o', label="data")
plt.plot(X_train, y_pred_train, 'o', label="prediction")
plt.plot([X.min(), X.max()], [min_pt, max_pt], label='fit')
plt.legend(loc='best')

正如上图所示，这条线能捕捉到数据的大致斜率，但是没多少细节。

接下来，试试测试集:

In [ ]:
y_pred_test = regressor.predict(X_test)

In [ ]:
plt.plot(X_test, y_test, 'o', label="data")
plt.plot(X_test, y_pred_test, 'o', label="prediction")
plt.plot([X.min(), X.max()], [min_pt, max_pt], label='fit')
plt.legend(loc='best');

同样，scikit-learn提供了``score``方法，可以定量评价预测结果。对于回归任务，得到的是R<sup>2</sup>得分。另一种流行的方法是均方误差。顾名思义，均方误差就是预测值和实际目标值的平均平方差

$$MSE = \frac{1}{n} \sum^{n}_{i=1} (\text{predicted}_i - \text{true}_i)^2$$

In [ ]:
regressor.score(X_test, y_test)

KNeighborsRegression
=======================
像分类一样，也可以用基于近邻的方法进行回归。可以简单地取最近点的输出，或者取几个最近点求平均值。这种方法在回归中不如在分类中受欢迎，但仍然是一个很好的基线方法。

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
kneighbor_regression = KNeighborsRegressor(n_neighbors=1)
kneighbor_regression.fit(X_train, y_train)

再次，让我们看看训练集和测试集的表现:

In [ ]:
y_pred_train = kneighbor_regression.predict(X_train)

plt.plot(X_train, y_train, 'o', label="data", markersize=10)
plt.plot(X_train, y_pred_train, 's', label="prediction", markersize=4)
plt.legend(loc='best');

在训练集上，表现非常好：每个点都是它自己最近邻！

In [ ]:
y_pred_test = kneighbor_regression.predict(X_test)

plt.plot(X_test, y_test, 'o', label="data", markersize=8)
plt.plot(X_test, y_pred_test, 's', label="prediction", markersize=4)
plt.legend(loc='best');

在测试集上，在捕捉变化方面也做得更好，但是估计结果看起来比以前更混乱。来看看R<sup>2</sup>得分:

In [ ]:
kneighbor_regression.score(X_test, y_test)

比以前好多了！这里，线性模型并不适合我们的问题；它缺少复杂性，与数据不相符。

练习
=========
在波士顿住房数据集上比较KNeighborsRegressor和LinearRegression。可以用``sklearn.datasets.load_boston``加载数据集，通过阅读``DESCR``属性了解数据集更多信息。

In [ ]:
# %load solutions/06A_knn_vs_linreg.py